
# Семинар 12 — Основы ООП (Python)

**Предпосылки:** базовый Python (функции, списки, словари)  
**Цели обучения:**
1. Понимать классы/объекты; использовать атрибуты и методы  
2. Различать данные экземпляра и класса; применять `@classmethod` / `@staticmethod`  
3. Инкапсулировать состояние через свойства (`@property`)  
4. Моделировать связи: **композиция** vs **наследование**  
5. Применять полиморфизм и «утиная типизация»  
6. Освоить 2 практичных паттерна (фабрика, стратегия)



## 1) Разминка
**Подумайте/обсудите:**  
- В чём разница между *классом* и *экземпляром*?  
- Когда предпочтительнее **композиция**, а не **наследование**?  
- Что обозначает `self`?


In [1]:

# Быстрая самопроверка
answers = {
    "class_vs_instance": "Класс — это чертёж (тип); экземпляр — конкретный объект этого типа.",
    "prefer_composition_when": "Когда нужна связь 'имеет' (has‑a) и хочется слабой связанности вместо наследования.",
    "self": "Текущий экземпляр, для которого вызывается метод (передаётся неявно)."
}
for k, v in answers.items():
    print(f"{k}: {v}")


class_vs_instance: Класс — это чертёж (тип); экземпляр — конкретный объект этого типа.
prefer_composition_when: Когда нужна связь 'имеет' (has‑a) и хочется слабой связанности вместо наследования.
self: Текущий экземпляр, для которого вызывается метод (передаётся неявно).



## 2) База: атрибуты и методы
Ключевые элементы: `class`, `__init__`, атрибуты экземпляра, `self`, `__repr__`.


In [23]:
class BankAccount:
    """Простой банковский счёт: пополнение/снятие и печать состояния."""
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance

    def deposit(self, amount):
        if amount < 0:
            raise ValueError("сумма должна быть >= 0")
        self.balance += amount

    def withdraw(self, amount):
        if amount < 0:
            raise ValueError("сумма должна быть >= 0")
        if amount > self.balance:
            raise ValueError("недостаточно средств")
        self.balance -= amount

    def __repr__(self):
        return f"BankAccount({self.owner!r}, balance={self.balance})"

# Демонстрация
alice = BankAccount("Алиса", 100)
alice.deposit(50)
try:
    alice.withdraw(200)
except ValueError as e:
    print("Ожидаемая ошибка:", e)
print(alice)


Ожидаемая ошибка: недостаточно средств
BankAccount('Алиса', balance=150)


https://www.w3schools.com/python/ref_func_setattr.asp

In [25]:

# УПРАЖНЕНИЕ: Реализуйте transfer_to(self, other, amount)
# - Сначала списать со своего счёта, затем зачислить на другой
# - Бросать ValueError, если other не BankAccount
# - При любой ошибке не менять балансы (простая "атомарность")

def _add_transfer_to():
    def transfer_to(self, other, amount):
        if not isinstance(other, BankAccount):
            raise ValueError("other должен быть BankAccount")
        src_balance, dst_balance = self.balance, other.balance
        try:
            self.withdraw(amount)
            other.deposit(amount)
        except Exception:
            self.balance, other.balance = src_balance, dst_balance
            raise
    setattr(BankAccount, "transfer_to", transfer_to)

_add_transfer_to()

# Тесты
a = BankAccount("A", 100)
b = BankAccount("B", 10)
a.transfer_to(b, 40)
assert a.balance == 60 and b.balance == 50, "перевод должен переносить средства"

try:
    a.transfer_to("не счёт", 10)
    assert False, "должно было быть исключение"
except ValueError:
    pass
print("✓ transfer_to работает")


✓ transfer_to работает



## 3) Данные класса vs экземпляра (+ static/class methods)
**Данные класса** общие для всех экземпляров. `@classmethod` работает с классом, `@staticmethod` — утилита без `self/cls`.


In [40]:
class Ticket:
    #_next_id = 1  # атрибут класса

    def __init__(self):
        self.id = Ticket._next_id
        Ticket._next_id += 1

    @classmethod
    def reset_ids(_):
        Ticket._next_id = 1


# Демонстрация
Ticket.reset_ids()
t1, t2 = Ticket(), Ticket()
assert t1.id == 1 and t2.id == 2, "идентификаторы должны увеличиваться"
t3 = Ticket()
assert t3.id == 3, "идентификаторы должны увеличиваться"
Ticket.reset_ids()
t4 = Ticket()
assert t4.id == 1, "после reset_ids() снова 1"
print("✓ Ticket: классовые/статические методы ок")


✓ Ticket: классовые/статические методы ок


In [41]:

class Ticket:
    _next_id = 1  # атрибут класса

    def __init__(self, title):
        if not self.is_valid_title(title):
            raise ValueError("заголовок не должен быть пустым")
        self.id = Ticket._next_id
        Ticket._next_id += 1
        self.title = title

    @classmethod
    def reset_ids(self):
        self._next_id = 1

    @staticmethod
    def is_valid_title(t):
        return bool(t and str(t).strip())

# Демонстрация
Ticket.reset_ids()
t1, t2 = Ticket("Баг логина"), Ticket("Добавить поиск")
assert t1.id == 1 and t2.id == 2, "идентификаторы должны увеличиваться"
t3 = Ticket("Снова первый")
assert t3.id == 3, "идентификаторы должны увеличиваться"
Ticket.reset_ids()
t4 = Ticket("Снова первый")
assert t4.id == 1, "после reset_ids() снова 1"
print("✓ Ticket: классовые/статические методы ок")


✓ Ticket: классовые/статические методы ок



## 4) Инкапсуляция со свойствами (`@property`)
Проверяйте и вычисляйте значения через `@property` и сеттеры.


In [48]:
class Value:
    def __init__(self, x):
        self.value = x

    @property
    def value(self):
        print("getter value")
        return self._value

    @value.setter
    def value(self, x):
        print("setter value")
        if x <= 0:
            raise ValueError("value должен быть > 0")
        self._value = x

v = Value(5)
assert v.value == 5, "getter value"

setter value
getter value


In [44]:

class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    @property
    def width(self):
        print("getter width")
        return self._width

    @width.setter
    def width(self, value):
        print("setter width")
        if value <= 0:
            raise ValueError("width должен быть > 0")
        self._width = value

    # Свойство height с валидацией аналогично width
    @property
    def height(self):
        print("getter height")
        return self._height

    @height.setter
    def height(self, value):
        print("setter height")
        if value <= 0:
            raise ValueError("height должен быть > 0")
        self._height = value

    @property
    def area(self):
        print("getter area")
        return self.width * self.height

# Тесты
r = Rectangle(3, 4)
assert r.area == 12, "area = width * height"
try:
    r.width = -1
    assert False, "отрицательная ширина должна приводить к ошибке"
except ValueError:
    print("Exception")
print("✓ Свойства Rectangle ок")


setter width
setter height
getter area
getter width
getter height
setter width
Exception
✓ Свойства Rectangle ок



## 5) Моделирование связей: композиция vs наследование
Для связи *имеет* — **композиция**; для *является* — **наследование**.


In [49]:

# Композиция
class Logger:
    def log(self, msg):
        print(f"[log] {msg}")

class Downloader:
    def __init__(self):
        self.logger = Logger()

    def fetch(self, url):
        self.logger.log(f"Загрузка {url}")
        return f"<содержимое {url}>"

d = Downloader()
_ = d.fetch("https://example.com")


[log] Загрузка https://example.com


In [57]:

# Наследование
from math import pi

class Shape:
    def info(self):
        print(f"Это Shape")

    def area(self):
        raise NotImplementedError

class Circle(Shape):
    def __init__(self, r):
        self.r = r

    def area(self):
        return pi * self.r**2

class Square(Shape):
    def __init__(self, s):
        self.s = s

    def area(self):
        return self.s**2

    def info(self):
      print(f"Это квадрат со стороной {self.s}")

shapes = [Circle(2), Square(3)]
print([round(s.area(), 2) for s in shapes])
print([s.info() for s in shapes])


[12.57, 9]
Это Shape
Это квадрат со стороной 3
[None, None]



**Обсуждение:** почему `Square(Circle)` — плохая идея? Квадрат — **не** круг; нарушится принцип подстановки Лисков.



## 6) Полиморфизм и «утиная» типизация
Одинаковый интерфейс — разные реализации. В Python важно не наследование, а наличие нужных методов.


In [59]:

def total_area(shapes):
    return sum(s.area() for s in shapes)

class FakeShape:
    def area(self):
        return 10

print("total:", total_area([Circle(1), Square(2), FakeShape()]))


total: 17.141592653589793



## 7) Два полезных паттерна
### A) Фабрика (factory)


In [68]:

class User:
    def __init__(self, username, email):
        self.username = username
        self.email = email

    @classmethod
    def from_dict(cls, d):
        return cls(d["username"], d["email"])

    @classmethod
    def from_email(cls, email):
      if "@" not in email:
        raise ValueError("некорректный email")
      username = email.split("@", 1)[0]
      return cls(username, email)

# Тесты
u = User.from_dict({"username":"lin", "email":"lin@example.com"})
u2 = User.from_email("ada@lovelace.org")
assert u.username == "lin"
assert u2.username == "ada" and u2.email.endswith("@lovelace.org")
print("✓ Фабрики User ок")


✓ Фабрики User ок



### B) Стратегия (strategy)
Заменяемое поведение за счёт композиции взаимозаменяемых объектов.


In [69]:

#class DiscountStrategy:
#    def apply(self, price):
#        return price

class PercentageOff:
    def __init__(self, pct):
        self.pct = pct
    def apply(self, price):
        return price * (1 - self.pct)

class FixedAmountOff:
    # Конструктор задаёт сумму скидки; итог не должен быть ниже 0
    def __init__(self, amount):
        self.amount = amount
    def apply(self, price):
        return max(0, price - self.amount)

class Cart:
    def __init__(self, prices, strategy):
        self.prices = list(prices)
        self.strategy = strategy
    def total(self):
        subtotal = sum(self.prices)
        return self.strategy.apply(subtotal)

# Тесты
cart = Cart([10, 20, 5], PercentageOff(0.1))
assert abs(cart.total() - 31.5) < 1e-6, "скидка 10% от 35 == 31.5"
cart2 = Cart([10, 5], FixedAmountOff(12))
assert cart2.total() == 3, "фиксированная скидка с нижней границей 0"
print("✓ Паттерн Strategy ок")


✓ Паттерн Strategy ок
